In [ ]:
import pickle

In [1]:
# Import Modules #########################################################
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.graph_objs as go
#import config
from   datetime import timedelta
from   datetime import datetime as dt
from   dash.dependencies import Input, Output, State

In [14]:
# Pre-Startup ############################################################
print("Initializing Dash and Loading Dataset...")
app = dash.Dash(__name__)
app.config['suppress_callback_exceptions'] = True

Initializing Dash and Loading Dataset...


In [3]:
mapbox_access_token = "pk.eyJ1Ijoia3RkMjAwMSIsImEiOiJjanUwZzBkZWYxYWdyNDRtcTB3M3Rqb2w2In0.HC_MMaor7bWiJEu7Ytp7pA"
map_data = df = pd.read_csv("20restuarnte.dat",parse_dates=True)
#df.head(10)


In [4]:
df["inspection_date"] = df["inspection_date"].str.replace('12:00:00 AM',' ')
#df["inspection_date"].head(10)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 17 columns):
business_id              19 non-null int64
business_name            19 non-null object
business_address         19 non-null object
business_city            19 non-null object
business_state           19 non-null object
business_postal_code     19 non-null int64
business_latitude        12 non-null float64
business_longitude       12 non-null float64
business_location        12 non-null object
business_phone_number    5 non-null float64
inspection_id            19 non-null object
inspection_date          19 non-null object
inspection_score         13 non-null float64
inspection_type          19 non-null object
violation_id             16 non-null object
violation_description    16 non-null object
risk_category            16 non-null object
dtypes: float64(4), int64(2), object(11)
memory usage: 2.6+ KB


In [6]:
df.isna()
unique_inspection_type_list       = df['inspection_type'].dropna().unique().tolist()
unique_risk_category_list         = df['risk_category'].dropna().unique().tolist()
unique_inspection_score_list      = df['inspection_score'].dropna().unique().tolist()

print(unique_inspection_type_list)
print('    ')
print(unique_risk_category_list)
print('    ')
print(unique_inspection_score_list)

['Complaint', 'Routine - Unscheduled', 'Reinspection/Followup', 'New Ownership']
    
['High Risk', 'Low Risk', 'Moderate Risk']
    
[92.0, 86.0, 84.0, 71.0, 93.0, 77.0, 72.0, 98.0, 81.0, 100.0]


In [7]:
start_date       = df['inspection_date'].max()
print(start_date)
end_date         = df['inspection_date'].max()
print(end_date)
min_date_allowed = df['inspection_date'].min()
print(min_date_allowed)

12/07/2017  
12/07/2017  
01/28/2019  


In [8]:
# Data Driven Widgets Initial Declarations ###############################
print("Declaring Data Driven Widgets...")
htmlh3_top_info                = html.H3('San Fransico Resturante Inspection Analysis')
dropdown_map_marker_color      = dcc.Dropdown(
                                    id =  "DROPDOWN_MAP_MARKER_COLOR",
                                    options = [
                                        { 'label': "Risk Category",    'value': 'risk_category' },
                                        { 'label': "Inspection Score", 'value': 'inspection_score' },
                                        { 'label': "Zip Code",         'value': 'busines_postal_code' }
                                    ],
                                    value = "Risk Category"
                                )
scattermapbox_the_map          = dcc.Graph(id="SCATTERMAPBOX_THE_MAP", config = { 'scrollZoom': True })
datepickerrange_inspected      = dcc.DatePickerRange(
                                     id               = "DATEPICKERRANGE_INSPECTED",
                                     start_date       = df['inspection_date'].max(),
                                     end_date         = df['inspection_date'].max(),
                                     min_date_allowed = df['inspection_date'].min()
                               )
htmlh4_latitude                = html.H4(id="HTMLH4_LATITUDE")
rangeslider_latitude           = dcc.RangeSlider(
                                    id="RANGESLIDER_LATITUDE",
                                    min   = df['business_latitude'].min(),
                                    max   = df['business_latitude'].max(),
                                    value = [ df['business_latitude'].min(), df['business_latitude'].max() ]
                               )

htmlh4_longitude               = html.H4(id="HTMLH4_LONGITUDE")
rangeslider_longitude          = dcc.RangeSlider(
                                    id="RANGESLIDER_LONGITUDE",
                                    min   = df['business_longitude'].min(),
                                    max   = df['business_longitude'].max(),
                                    value = [ df['business_longitude'].min(), df['business_longitude'].max() ]
                               )

htmlh4_zip_code                = html.H4(id="HTMLH4_ZIP_CODE")
rangeslider_zip_code           = dcc.RangeSlider(
                                    id="RANGESLIDER_ZIP_CODE",
                                    min   = df['business_postal_code'].min(),
                                    max   = df['business_postal_code'].max(),
                                    value = [ df['business_postal_code'].min(), df['business_postal_code'].max() ]
                               )

htmlh4_inspection_score        = html.H4(id="HTMLH4_INSPECTION_SCORE")
rangeslider_inspection_score   = dcc.RangeSlider(
                                    id="RANGESLIDER_INSPECTION_SCORE",
                                    min   = df['inspection_score'].min(),
                                    max   = df['inspection_score'].max(),
                                    value = [ df['inspection_score'].min(), df['inspection_score'].max() ]
                               )
checklist_inspection_type       = dcc.Checklist(
                                    id="CHECKLIST_INSPECTION_TYPE",     
                                    options = [{'label': i, 'value': i} for i in sorted(unique_inspection_type_list)],     
                                    values = unique_inspection_type_list)
checklist_risk_category         = dcc.Checklist(
                                    id="CHECKLIST_RISK_CATEGORY", 
                                    options = [{'label': i, 'value': i} for i in sorted(unique_risk_category_list)], 
                                    values = unique_risk_category_list)

Declaring Data Driven Widgets...


In [9]:
# GUI Input Dependencies #################################################
print("Declaring Which Input Widgets Trigger Display Updates")
inputs_which_trigger_updates = [
    Input("DATEPICKERRANGE_INSPECTED", "start_date"),
    Input("DATEPICKERRANGE_INSPECTED", "end_date"),
 
    Input("RANGESLIDER_ZIP_CODE", "value"),
    Input("RANGESLIDER_LATITUDE", "value"),
    Input("RANGESLIDER_LONGITUDE", "value"),
 
    Input("CHECKLIST_INSPECTION_TYPE", "values"),
    Input("CHECKLIST_RISK_CATEGORY", "values"),
    Input("RANGESLIDER_INSPECTION_SCORE", "value")
]

Declaring Which Input Widgets Trigger Display Updates


In [10]:
# Layout #################################################################
print("Declaring Application Layout...")
app.layout = html.Div([
    html.Div([htmlh3_top_info, dropdown_map_marker_color], id = 'TOP_INFO'),
 
    html.Div([
        html.Div([
            scattermapbox_the_map
        ], id="MAP_CONTAINER")
    ], id='VISUAL_VIEWS_CONTAINER'),
 
    html.Div([
      html.H2("When"),
      html.Div([
          html.H4("Inspection Date: "), datepickerrange_inspected
      ], id="WHEN_CONTAINER"),
 
      html.H2("Where"),
      html.Div([
          htmlh4_zip_code,  rangeslider_zip_code,
          htmlh4_latitude,  rangeslider_latitude,
          htmlh4_longitude, rangeslider_longitude
      ], id="WHERE_CONTAINER"),
 
      html.H2("Violation"),
      html.Div([
          html.H4("Inspection Type: "), checklist_inspection_type,
          html.H4("Risk Category: "), checklist_risk_category,
          htmlh4_inspection_score, rangeslider_inspection_score
      ], id="VIOLATION_CONTAINER")
    ], id='FILTER_INPUTS_CONTAINER')
], id="MAIN_CONTAINER")


Declaring Application Layout...


In [11]:
# UI Callbacks ###########################################################
@app.callback(Output("HTMLH4_ZIP_CODE", "children"), [ Input("RANGESLIDER_ZIP_CODE", "value") ])
def update_zip_code_label(selected_range):
    return "Zip Code: %d to %d" % (selected_range[0], selected_range[1])
 
@app.callback(Output("HTMLH4_LATITUDE", "children"), [ Input("RANGESLIDER_LATITUDE", "value") ])
def update_latitude_label(selected_range):
    return "Latitude: From %.4f to %.4f" % (selected_range[0], selected_range[1])
 
@app.callback(Output("HTMLH4_LONGITUDE", "children"), [ Input("RANGESLIDER_LONGITUDE", "value") ])
def update_longitude_label(selected_range):
    return "Longitude: From %.4f to %.4f" % (selected_range[0], selected_range[1])
 
@app.callback(Output("HTMLH4_INSPECTION_SCORE", "children"), [ Input("RANGESLIDER_INSPECTION_SCORE", "value") ])
def update_inspection_score_label(selected_range):
    return "Inspection Score: From %d to %d" % (selected_range[0], selected_range[1])
 
@app.callback(Output("SCATTERMAPBOX_THE_MAP", "figure"), [
    Input("DROPDOWN_MAP_MARKER_COLOR", "value"),
    Input("DATEPICKERRANGE_INSPECTED", "start_date"),
    Input("DATEPICKERRANGE_INSPECTED", "end_date"),
 
    Input("RANGESLIDER_ZIP_CODE", "value"),
    Input("RANGESLIDER_LATITUDE",  "value"),
    Input("RANGESLIDER_LONGITUDE", "value"),
 
    Input("CHECKLIST_INSPECTION_TYPE", "values"),
    Input("CHECKLIST_RISK_CATEGORY", "values"),
    Input("RANGESLIDER_INSPECTION_SCORE", "value")
])
def when_triggers_update_scattermapbox(
    map_marker_color,
    inspected_start_date,
    inspected_end_date,
    zip_code_range,
    latitude_range,
    longitude_range,
    violation_inspection_type,
    violation_risk_category,
    violation_inspection_score
):
    print(map_marker_color)
    sf = df[
        (df['Inspected Date'].between(pd.to_datetime(start_date), pd.to_datetime(end_date), inclusive = True))
        &
        (df['Zip Code'].between(zip_code_range[0], zip_code_range[1], inclusive=True))
        &
        (df['Latitude'].between(latitude_range[0], latitude_range[1], inclusive=True))
        &
        (df['Longitude'].between(longitude_range[0], longitude_range[1], inclusive=True))
        &
        (df['Violation Score'].between(victim_age_range[0], victim_age_range[1], inclusive=True))
        &
        (df['Violation Inspection Type'].isin(violation_inspection_type))
        &
        (df['Violation Risk Category'].isin(violation_risk_category))
        &
        (~df["business_name"].isnull())
        &
        (~df["business_address"].isnull())
        &
        (~df["violation_description"].isnull())
    ]
    marker_label = generate_marker_labels(sf)
 
    map_marker_views = {
        'Inspection Date': {
            'colorscale': [
                [ 0.0, 'rgb(0, 0, 0)'     ],
                [ 0.1, 'rgb(128, 64, 0)' ],
                [ 0.3, 'rgb(255, 255, 0)' ],
                [ 0.5, 'rgb(0, 0, 255)'   ],
                [ 0.7, 'rgb(255, 128, 0)' ],
                [ 0.9, 'rgb(128, 64, 0)' ],
                [ 1.0, 'rgb(0, 0, 0)'     ]
            ],
            'cmin': 2359,
            'cmax': 0,
            'nticks': 24
        },
 
        'Inspection Score': {
            'colorscale': 'Viridis',
            'cmin': sf['Inspection Score'].min(),
            'cmax': sf['Inspection Score'].max(),
            'nticks': 30
        }
    }
 
    return go.Figure(
        data = [
            go.Scattermapbox(
                lat  = sf.Latitude,
                lon  = sf.Longitude,
                mode = 'markers',
                marker = go.scattermapbox.Marker(
                    size = 6,
                    color = sf[map_marker_color],
                    colorscale = map_marker_views[map_marker_color]["colorscale"],
                    cmin       = map_marker_views[map_marker_color]["cmin"],
                    cmax       = map_marker_views[map_marker_color]["cmax"],
                    colorbar = dict(
                        title  = map_marker_color,
                        nticks = map_marker_views[map_marker_color]["nticks"]
                    )
                ),
                text = marker_label
            )
        ],
 
        layout = go.Layout(
            height = 600,
            mapbox = go.layout.Mapbox(
                mapbox_access_token = config.mapbox_access_token,
                bearing     = 0,
                center = go.layout.mapbox.Center(
                    lat = 37.74,
                    lon = -122.3
                ),
               pitch = 45,
               zoom = 9
            )
        )
    )
 
# Help Function ##########################################################
def generate_marker_labels(sf):
    marker_label  =       "Inspection ID #: " + sf["inspection_id"].astype(str)
    marker_label += "<br>    Business Name: " + sf["business_name"]
    marker_label += "<br> Business Address: " + sf["business_address"].astype(str)
    marker_label += "<br> Inspection Score: " + sf["inspection_score"].astype(str)
    marker_label += "<br>  Inspection Type: " + sf["inspection_type"].astype(str)
    marker_label += "<br>        Violation: " + sf["violation_description"]
    marker_label += "<br>    Risk Category: " + sf["risk_category"]
 
    return marker_label
 
# Server Startup #########################################################
if __name__ == "__main__":
    app.run_server(debug=True)

Running on http://127.0.0.1:8050/
Debugger PIN: 913-556-543


SystemExit: 1

/home/ktd2001/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

